<a href="https://colab.research.google.com/github/YahiaML/Scraping-a-database-from-Udacity-page/blob/main/Scraping_a_database_from_Udacity_page.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import libraries

In [110]:
from bs4 import BeautifulSoup
import requests
import sqlite3
import re 
import pandas as pd

# Scraping the database bage

In [100]:
url  = "https://video.udacity-data.com/topher/2020/May/5eb5533b_parch-and-posey/parch-and-posey.sql"
response = requests.get(url)
data = response.text

In [101]:
Block_of_lines = BeautifulSoup(data,"html.parser")

In [102]:
# I find that the lines are treated as 1 block of lines, so I needed to split them
lines = re.split(";", str(Block_of_lines)) 

# **Gathering the database in .db format (Attempt #1)**

In [103]:
#creating an empty data base neamed p_and_p_db
p_and_p_db = sqlite3.connect('p_and_p_db.db')

In [104]:
for line in lines:
    p_and_p_db.execute(line + ";") 

OperationalError: ignored

I found that this code can not gather the data base due to unknown error

# **Gathering the database in .db format (Attempt #2)**

Let us make a simple try and except

In [105]:
for line in lines:
  try:
    p_and_p_db.execute(line + ";")
  except Exception as e:
    print(e)

# This code did gather the data base but it has missing values

cannot start a transaction within a transaction
table web_events already exists
table sales_reps already exists
table region already exists
table orders already exists
table accounts already exists
unrecognized token: "'AT&amp;"
near "T": syntax error
unrecognized token: "'Procter &amp;"
near "Gamble": syntax error
unrecognized token: "'Johnson &amp;"
near "Johnson": syntax error
unrecognized token: "'PG&amp;"
near "E": syntax error
unrecognized token: "'Goodyear Tire &amp;"
near "Rubber": syntax error
unrecognized token: "'Marsh &amp;"
near "McLennan": syntax error
unrecognized token: "'Bed Bath &amp;"
near "Beyond": syntax error
unrecognized token: "'R.R. Donnelley &amp;"
near "Sons": syntax error
unrecognized token: "'Stanley Black &amp;"
near "Decker": syntax error
unrecognized token: "'BB&amp;"
near "T": syntax error
unrecognized token: "'Air Products &amp;"
near "Chemicals": syntax error
unrecognized token: "'Owens &amp;"
near "Minor": syntax error
unrecognized token: "'Reliance 

This code did gather the data base but it has missing values

# **Gathering the database in .db format (Attempt #3)**

Let us see what these errors really are

In [106]:
parch_and_posey_1 = sqlite3.connect('parch_and_posey_1.db')

In [107]:
for line in lines:
  try:
    parch_and_posey_1.execute(line + ";")
  except Exception as e:
    print("EROR", e)
    print()
    print("LINE:", line)
    print()
    print()

EROR unrecognized token: "'AT&amp;"

LINE: 
INSERT INTO accounts VALUES (1091,'AT&amp


EROR near "T": syntax error

LINE: T','www.att.com',42.4974627,-74.90271225,'Jamel Mosqueda',321590)


EROR unrecognized token: "'Procter &amp;"

LINE: 
INSERT INTO accounts VALUES (1331,'Procter &amp


EROR near "Gamble": syntax error

LINE:  Gamble','www.pg.com',41.33403574,-74.18997513,'Cordia Winters',321620)


EROR unrecognized token: "'Johnson &amp;"

LINE: 
INSERT INTO accounts VALUES (1381,'Johnson &amp


EROR near "Johnson": syntax error

LINE:  Johnson','www.jnj.com',40.6266526,-75.1543044,'Genevieve Fenton',321670)


EROR unrecognized token: "'PG&amp;"

LINE: 
INSERT INTO accounts VALUES (2651,'PG&amp


EROR near "E": syntax error

LINE: E Corp.','www.pge.com',33.81262481,-84.3145316,'Whitney Koga',321810)


EROR unrecognized token: "'Goodyear Tire &amp;"

LINE: 
INSERT INTO accounts VALUES (2681,'Goodyear Tire &amp


EROR near "Rubber": syntax error

LINE:  Rubber','www.goodyear.com',33.

After checking https://video.udacity-data.com/topher/2020/May/5eb5533b_parch-and-posey/parch-and-posey.sql and searching for the corrabted lines, I found that:

1. This error occured due to unknown reason in splitting lines after & sign in names in account table.

2. The first part of the line rises an error "unrecognized token".

3. The second part of the line rises an error "syntax error".

So, what I'm going to do is just join the two parts again in one line and run the code normally

In [108]:
parch_and_posey = sqlite3.connect('parch_and_posey.db')

text = ""
for line in lines:
  try:
    parch_and_posey.execute(line + ";")
  except Exception as e:
    error_msj = str(e)
    if "unrecognized token" in error_msj:
      text += line
    elif "syntax error" in error_msj:
      text += line
      parch_and_posey.execute(text + ";")
      text = ""

    else:
      print("EROR", e)
      print()
      print("LINE:", line)
      print()

No error, HOORAY

#**Now let's try the database**

In [ ]:
# print output as rows
"""
cursor = parch_and_posey.execute(''' SELECT *                            
                          FROM accounts 
                          ''')

for row in cursor:
  print(row)    """

In [111]:
output = parch_and_posey.execute(''' SELECT *                            
                          FROM accounts 
                          ''')

# print output as a table (data frame)
data = []
for row in output:
  data.append(list(row))
df= pd.DataFrame(data,columns=["Id","Name", "website", "lat", "long ", "primary_poc", "sales_rep_id "])  # inset columns names that you want
df

,Id,Name,website,lat,long,primary_poc,sales_rep_id
0,1001,Walmart,www.walmart.com,40.238496,-75.103297,Tamara Tuma,321500
1,1011,Exxon Mobil,www.exxonmobil.com,41.169156,-73.849374,Sung Shields,321510
2,1021,Apple,www.apple.com,42.290495,-76.084009,Jodee Lupo,321520
3,1031,Berkshire Hathaway,www.berkshirehathaway.com,40.949021,-75.763898,Serafina Banda,321530
4,1041,McKesson,www.mckesson.com,42.217093,-75.284998,Angeles Crusoe,321540
...,...,...,...,...,...,...,...
346,4461,KKR,www.kkr.com,45.545353,-122.655247,Buffy Azure,321970
347,4471,Oneok,www.oneok.com,45.513513,-122.681500,Esta Engelhardt,321960
348,4481,Newmont Mining,www.newmont.com,45.494117,-122.669460,Khadijah Riemann,321970
349,4491,PPL,www.pplweb.com,45.491720,-122.671880,Deanne Hertlein,321960
